In [ ]:
import tensorflow as tf
import numpy as np
import os
import os.path
import transfer_learning  # This module contains large amounts of pre-written code

In [ ]:
# Ensure target log dir exists
INCEPTION_LOG_DIR = './tmp/inception_v3_log'
if not os.path.exists(INCEPTION_LOG_DIR):
    os.makedirs(INCEPTION_LOG_DIR)

In [ ]:
# Ensure set Right dir, it will randomly select 10% of the trainning image as validate images
training_images, testing_images, label_maps = transfer_learning.create_image_lists('../data/train', testing_percentage=10, max_number_images=25005)

In [ ]:
# create model, save at './model'
graph, bottleneck, resized_input, softmax = transfer_learning.create_model()

In [ ]:
# compute single bottleneck of a image
def compute_bottleneck(session, image_data):
    
    bottleneck_output= session.run((bottleneck),{'DecodeJpeg/contents:0': image_data})
    
    return(bottleneck_output)

In [ ]:
#compute all the bottlenecks from training image list and save at'./data/bottleneck'
with graph.as_default():
    with tf.Session() as session:
        transfer_learning.cache_bottlenecks(compute_bottleneck, session, training_images)

In [ ]:
# make last layer
def make_final_layers(bottleneck_tensor, num_classes):

    bottleneck_tensor_size = int(bottleneck.shape[1])
    
    with tf.variable_scope('input'):
        bottleneck_input = tf.placeholder_with_default(
            bottleneck_tensor,
            [None, bottleneck_tensor_size],
            'bottleneck_input')
        
        label_input = tf.placeholder(tf.int64, [None], name='label_input')
    
    logits = tf.layers.dense(bottleneck_input, num_classes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=label_input, logits=logits)
    loss_summary = tf.summary.scalar('cross_entropy', loss)
    optimizer = tf.train.GradientDescentOptimizer(0.1)
    train_step = optimizer.minimize(loss)
    pred_prob = tf.nn.softmax(logits)

    
    return bottleneck_input, label_input, logits, train_step, loss_summary , pred_prob

In [ ]:
#compute accuracy of input image_list
def compute_accuracy(labels, logits):

    prediction = tf.argmax(logits,1, name='pred_class')
    #true_label = tf.argmax(labels, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, labels), tf.float32))
    accuracy_summary=tf.summary.scalar('accuracy', accuracy)

    return prediction, accuracy, accuracy_summary

In [ ]:
# adding layer to graph

with graph.as_default():
    bottleneck_input, label_input, logits, train_step, loss_summary , pred_prob = make_final_layers(bottleneck, len(label_maps))
    prediction, accuracy, accuracy_summary = compute_accuracy(label_input, logits)
    summary_op = tf.summary.merge([loss_summary, accuracy_summary])
    

In [ ]:
def execute_train_step(session: tf.Session, summary_writer: tf.summary.FileWriter, current_step: int):

    _, ac, summary = session.run((train_step, accuracy, summary_op),
                       feed_dict={bottleneck_input: training_data_set['bottlenecks'],
                                  label_input: training_data_set['labels']
                                 })
    
    summary_writer.add_summary(summary, current_step)
    
    if current_step % 10 == 0:
        print('Accuracy at step {0} is {1}'.format(current_step, ac))
        

In [ ]:
# evaluate validate data set
def evaluate_images(session: tf.Session, images_jpeg_data: [bytes], labels: [int]):

    correct = []
    pred=[]
    
    for label, jpeg in zip(labels, images_jpeg_data):
        image_data = session.run(decoded_image, feed_dict={jpeg_data: jpeg})
        pd, ac = session.run((prediction,accuracy), feed_dict={resized_input: image_data, label_input: [label]})
        correct.append(ac)
        pred.append(pd)
    
    return np.mean(correct),pred

In [ ]:
# transform image path to byte like image
training_data_set = transfer_learning.create_training_dataset(training_images)

In [ ]:
with graph.as_default():
    jpeg_data, decoded_image = transfer_learning.make_jpeg_decoding()

In [ ]:
# get class probability of input image without labels
def get_prob(session: tf.Session, images_jpeg_data: [bytes]):

    pred = np.array([],dtype=np.float32).reshape(0,2)
    #name=np.array([],dtype=np.str).reshape(0,1)
    
    for jpeg in images_jpeg_data:
        image_data = session.run(decoded_image, feed_dict={jpeg_data: jpeg})
        pd = session.run(pred_prob, feed_dict={resized_input: image_data})
        pred = np.concatenate((pred,pd), axis = 0)
        #name = np.concatenate
    
    return pred

In [ ]:
#Train & Validate & Do test

with graph.as_default():
    with tf.Session() as session:
        print('------------- Starting training ----------------')
        
        session.run(tf.global_variables_initializer())
        
        summary_writer = tf.summary.FileWriter(os.path.join(INCEPTION_LOG_DIR, 'retrained'), graph)
        
        for i in range(100):
            execute_train_step(session, summary_writer, i)
        summary_writer.close()  
        
        print('------------- Training done! -------------------')
        print('---------- Loading testing data ----------------')
        
        tlabels, timages = transfer_learning.get_testing_data(testing_images)
        
        
        print('----------- Evaluating on Validating --------------')
        
        validate_result = evaluate_images(session, timages, tlabels)
        eval_accuracy = validate_result[0]
        valid_pred = validate_result[1]
        
        print('Evaluation accuracy was: {0}'.format(eval_accuracy))
        
        
        print('----------- Doing Testing--------------')  
        testing, _, _ = transfer_learning.create_image_lists('../data/test', testing_percentage=0, max_number_images=13000)
        _, testimages = transfer_learning.get_testing_data(testing)
        
        test_prob_sess= get_prob(session, testimages)
        
        np.save('test_prob_sess.npy', test_prob_sess, allow_pickle=False) 
        

In [ ]:
import re
import csv
import pandas as pd

names = [re.search('[0-9]+',i[1]).group() for i in testing]
result_sess = np.column_stack((names,test_prob_sess))
colnames = ['id','cat','dog']
result_sess_pd = pd.DataFrame(result_sess, index=names, columns=colnames)
result_sess_pd.to_csv('inception_full_sess_result.csv',index=False,header=True)
